# Exercise: Break a RFID Authentication Protocol

**Goal: Try to break RFID authentication protocol.**

**An attack is considered if keys K1, K2, ID are found and extract all of their bits.**
- Approach is correct if more than half (avg) is predicted, i.e., if far from nbits/2
- dH (distancia Hamming) to check accuracy of predictions: attack must outperform results by random
- Initially assume reader, tag, k1, k2, id and compute table for all combinations in ABDEF

In [2]:
import numpy as np
import pandas as pd

## David-Prasad protocol implementation
RFID ultra‐lightweight authentication protocol

In [3]:
results = {}
avgs = np.zeros((31,3))
iterations = 50
length = 96

# Constants K1, K2 and ID
Pid2 = np.random.binomial(n=1, p=0.5, size=length)
K1 = np.random.binomial(n=1, p=0.5, size=length)
K2 = np.random.binomial(n=1, p=0.5, size=length)
ID = np.random.binomial(n=1, p=0.5, size=length)

variables = [K1, K2, ID]

print("K1:", K1)
print("\nK2:", K2)
print("\nID:", ID)

K1: [1 1 1 0 1 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 0 0 1
 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1
 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 0 1]

K2: [1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1
 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0
 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0]

ID: [0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1
 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0
 0 0 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1]


In [4]:
# For section 2: 
def rotate(input, d):
   """ Slice string in two parts for left and right."""
   Lfirst = input[0:d]
   Lsecond = input[d:]
   
   return np.concatenate([Lsecond, Lfirst])

def hamming2(v1, v2):
    """ Hamming distance."""
    assert len(v1) == len(v2)
    value = sum(v1 != v2 for v1, v2 in zip(v1, v2))
    return value

In [5]:
for it in range(iterations):
  
  # Variables n1, n2 associated to specific session
  n1 = np.random.binomial(n=1, p=0.5, size=length) 
  n2 = np.random.binomial(n=1, p=0.5, size=length)

  # TAG verifies READER's identity and reads (n1, n2) specific of session 
  # READER sends A, B, D to TAG, and TAG locally obtains n1,n2 to check legitimacy
  A = (Pid2 & K1 & K2) ^ n1 
  B = ((~Pid2+2) & K1 & K2) ^ n2
  D = (K1 & n2) ^ (K2 & n1) 
  
  # READER verifies TAG's identity ("mutual authentication")
  # READER locally finds E, F (it knows all parameters),
  # and if it matches with which is received from TAG, authenticates 
  E = (K1 ^ n1 ^ ID) ^ (K2 & n2)
  F = (K1 & n1) ^ (K2 & n2)
  
  #F=(rotate(K1,3) & n1) ^ (rotate(K2,5) & n2)

  # Pid2 pseudonym is updated
  Pid2 = Pid2 ^ n1 ^ n2
  #print('A=',A,'\n','B=',B,'\n','D=',D,'\n','E=',E,'\n','F=',F,'\n')

  # ATTACK: Find keys K1, K2, ID and extract all of their bits
  # Approach is correct if more than half (avg) is predicted, i.e., if far from nbits/2
  # dH (distancia Hamming) to check accuracy of predictions: attack must outperform results by random
  # Initially assume reader, tag, k1, k2, id and compute table for all combinations in ABDEF

# All possible combinations
  L = [A, B, D, E, F, A^B, A^D, A^E, A^F, B^D, B^E, B^F, D^E, D^F, E^F, 
      A^B^D, A^B^E, A^B^F, A^D^E, A^D^F, A^E^F, B^D^E, B^D^F, B^E^F, D^E^F, 
      A^B^D^E, A^B^D^F, A^B^E^F, A^D^E^F, B^D^E^F, A^B^D^E^F]

  rows = ["A", "B", "D", "E", "F", "A^B", "A^D", "A^E", "A^F", "B^D", "B^E", "B^F", 
          "D^E", "D^F", "E^F", "A^B^D", "A^B^E", "A^B^F", "A^D^E", "A^D^F", "A^E^F",
          "B^D^E", "B^D^F", "B^E^F", "D^E^F", "A^B^D^E", "A^B^D^F", "A^B^E^F",
          "A^D^E^F", "B^D^E^F", "A^B^D^E^F"]
  
  results[it] = pd.DataFrame(columns=['dH(L,K1)', 'dH(L,K2)', 'dH(L,ID)'], 
                             index=rows)
   
  # Compute dH for each combination
  for i in L:
    values ={'dH(L,K1)': hamming2(i, variables[0]),
             'dH(L,K2)': hamming2(i, variables[1]),
             'dH(L,ID)': hamming2(i, variables[2])}
    results[it].fillna(value=values, limit=1, inplace=True)

  # Generate Hamming avgs for iterations performed
  avgs = np.add(avgs, results[it])

avgs = avgs / iterations
display(avgs)

# Best parameters to "break" the protocol
Bmejores = abs(avgs - (len(ID) / 2))
display(Bmejores)

,"dH(L,K1)","dH(L,K2)","dH(L,ID)"
A,47.90,47.18,46.94
B,48.42,47.30,49.34
D,38.00,37.64,46.52
E,47.72,48.56,48.56
F,38.56,37.08,46.68
A^B,47.60,47.32,47.20
A^D,35.94,60.42,49.30
A^E,49.10,43.18,50.06
A^F,60.06,36.30,50.14
B^D,60.38,36.62,51.22


,"dH(L,K1)","dH(L,K2)","dH(L,ID)"
A,0.10,0.82,1.06
B,0.42,0.70,1.34
D,10.00,10.36,1.48
E,0.28,0.56,0.56
F,9.44,10.92,1.32
A^B,0.40,0.68,0.80
A^D,12.06,12.42,1.30
A^E,1.10,4.82,2.06
A^F,12.06,11.70,2.14
B^D,12.38,11.38,3.22


In [6]:
# Extract indexes of 9 best results
mk1=Bmejores.nlargest(9, ['dH(L,K1)']).index
mk2=Bmejores.nlargest(9, ['dH(L,K2)']).index
mID=Bmejores.nlargest(9, ['dH(L,ID)']).index
#print(mk1)

# Save in lists those indexes for each metric (same positions as in L)
indexmk1, indexmk2, indexmID, signomk1, signomk2, signoID = [], [], [], [], [], []

In [7]:
for i in range(0, 9):
  
  v1=rows.index(mk1[i])
  indexmk1.append(v1)
  
  if avgs.iloc[v1, 0] < (len(ID) / 2):
    signomk1.append(1)
  else:
    signomk1.append(0)

  v2=rows.index(mk2[i])
  indexmk2.append(v2)
  
  if avgs.iloc[v2, 1] < (len(ID) / 2):
    signomk2.append(1)
  else:
    signomk2.append(0)

  v3=rows.index(mID[i])
  indexmID.append(v3)
  
  if avgs.iloc[v3, 2] < (len(ID) / 2):
    signoID.append(1)
  else:
    signoID.append(0)

print("Indexes of best combinations for K1: ", indexmk1)
print("Indexes of best combinations for K2: ", indexmk2)
print("Indexes of best combinations for ID: ", indexmID)

# To know which complementaries are needed
print(signomk1)
print(signomk2)
print(signoID)


Indexes of best combinations for K1:  [17, 15, 9, 6, 8, 11, 2, 4, 16]
Indexes of best combinations for K2:  [15, 17, 11, 6, 8, 9, 4, 2, 14]
Indexes of best combinations for ID:  [16, 14, 20, 12, 30, 21, 24, 29, 25]
[1, 1, 0, 1, 0, 1, 1, 1, 0]
[1, 1, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 1, 0, 1, 1, 1, 0, 0]


### Create DFs of matrixes with the 9 best combinations for K1, K2, ID

To predict K1

In [8]:
predictedK1, predictedK2, predictedID = [], [], []

matrizK1 = [L[indexmk1[0]], L[indexmk1[1]], L[indexmk1[2]],
            L[indexmk1[3]], L[indexmk1[4]], L[indexmk1[5]],
            L[indexmk1[6]], L[indexmk1[7]], L[indexmk1[8]]]

for a in range(len(indexmk1)):
  if signomk1[a]==0:
    matrizK1[a]=~matrizK1[a]+2
    
dfK1 = pd.DataFrame(matrizK1)

for c in dfK1.columns.values:
  moda=dfK1.iloc[:,c].value_counts().idxmax()
  predictedK1.append(moda)

To predict K2

In [9]:
matrizK2 = [L[indexmk2[0]], L[indexmk2[1]], L[indexmk2[2]],
            L[indexmk2[3]], L[indexmk2[4]], L[indexmk2[5]],
            L[indexmk2[6]], L[indexmk2[7]], L[indexmk2[8]]]

for a in range(len(indexmk2)):
  if signomk2[a]==0:
    matrizK2[a]=~matrizK2[a]+2
    
dfK2 = pd.DataFrame(matrizK2)
#display(dfK2)

for c in dfK2.columns.values:
  moda=dfK2.iloc[:,c].value_counts().idxmax()
  predictedK2.append(moda)

To predict ID

In [10]:
matrizID = [L[indexmID[0]], L[indexmID[1]], L[indexmID[2]],
            L[indexmID[3]], L[indexmID[4]], L[indexmID[5]],
            L[indexmID[6]], L[indexmID[7]], L[indexmID[8]]]

for a in range(len(indexmID)):
  if signoID[a]==0:
    matrizID[a]=~matrizID[a]+2
dfID = pd.DataFrame(matrizID)
#display(dfID)

for c in dfID.columns.values:
  moda=dfID.iloc[:,c].value_counts().idxmax()
  predictedID.append(moda)

### Our predictions...

In [11]:
print("Predicted K1: ", " ".join(map(str, predictedK1)))
print("K1 original: ", " ".join(map(str, K1)))
print("\nPredicted K2: ", " ".join(map(str, predictedK2)))
print("K2 original: ", " ".join(map(str, K2)))
print("\nPredicted ID: ", " ".join(map(str, predictedID)))
print("ID original: ", " ".join(map(str, ID)))

Predicted K1:  1 1 0 0 1 0 1 0 1 0 1 0 0 1 1 1 0 1 1 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1
K1 original:  1 1 1 0 1 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 0 1

Predicted K2:  1 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 0 1
K2 original:  1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0

Predicted ID:  0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1

### Evaluating our predictions
To aggregare more matrixes/sessions is needed to be more accurate

In [12]:
print(hamming2(K1, predictedK1))
print(hamming2(K2, predictedK2))
print(hamming2(ID, predictedID))

20
25
11


## Improve predictions...

In [13]:
predictedK1, predictedK2, predictedID = [], [], []

for i in range (100):
  
  # Variables n1, n2 associated to specific session
  n1 = np.random.binomial(n=1, p=0.5, size=length) 
  n2 = np.random.binomial(n=1, p=0.5, size=length)

  # TAG verifies READER's identity
  A = (Pid2 & K1 & K2) ^ n1 
  B = ((~Pid2+2) & K1 & K2) ^ n2
  D = (K1 & n2) ^ (K2 & n1) 
  
  # READER verifies TAG's identity ("mutual authentication")
  E = (K1 ^ n1 ^ ID) ^ (K2 & n2)
  F = (K1 & n1) ^ (K2 & n2)

  #F=(rotate(K1,3) & n1) ^ (rotate(K2,5) & n2)
    
  Pid2 = Pid2 ^ n1 ^ n2

  L = [A, B, D, E, F, A^B, A^D, A^E, A^F, B^D, B^E, B^F, D^E, D^F, E^F, 
      A^B^D, A^B^E, A^B^F, A^D^E, A^D^F, A^E^F, B^D^E, B^D^F, B^E^F, D^E^F, 
      A^B^D^E, A^B^D^F, A^B^E^F, A^D^E^F, B^D^E^F, A^B^D^E^F]

  # To predict K1: place in rows the results of the 9 best combinations
  matrizK1 = [L[indexmk1[0]], L[indexmk1[1]], L[indexmk1[2]],
              L[indexmk1[3]], L[indexmk1[4]], L[indexmk1[5]],
              L[indexmk1[6]], L[indexmk1[7]], L[indexmk1[8]]]
  
  for a in range(len(indexmk1)):
    if signomk1[a] == 0:
      matrizK1[a] = ~matrizK1[a] + 2 #complementary if needed
      
  dfK1aux = pd.DataFrame(matrizK1)
  frames= [dfK1aux,dfK1]
  dfK1= pd.concat(frames) #aggregate all k1 matrixes

  # To predict K2
  matrizK2 = [L[indexmk2[0]], L[indexmk2[1]], L[indexmk2[2]],
              L[indexmk2[3]], L[indexmk2[4]], L[indexmk2[5]],
              L[indexmk2[6]], L[indexmk2[7]], L[indexmk2[8]]]
  
  for a in range(len(indexmk2)):
    if signomk2[a] == 0:
      matrizK2[a] = ~matrizK2[a] + 2
      
  dfK2aux = pd.DataFrame(matrizK2)
  frames2= [dfK2aux,dfK2]
  dfK2= pd.concat(frames2)

  # To predict ID
  matrizID = [L[indexmID[0]], L[indexmID[1]], L[indexmID[2]],
              L[indexmID[3]], L[indexmID[4]], L[indexmID[5]],
              L[indexmID[6]], L[indexmID[7]], L[indexmID[8]]]
  
  for a in range(len(indexmID)):
    if signoID[a] == 0:
      matrizID[a]= ~matrizID[a] + 2
      
  dfIDaux = pd.DataFrame(matrizID)
  frames3= [dfIDaux,dfID]
  dfID= pd.concat(frames3)
#display(dfID)

In [14]:
# Extract mode of each column by voting (1 or 0)
for c in dfK1.columns.values:
  moda=dfK1.iloc[:,c].value_counts().idxmax()
  predictedK1.append(moda)
  
for c in dfK2.columns.values:
  moda=dfK2.iloc[:,c].value_counts().idxmax()
  predictedK2.append(moda)
  
for c in dfID.columns.values:
  moda=dfID.iloc[:,c].value_counts().idxmax()
  predictedID.append(moda) 

### Our final predictions of K1, K2, ID

In [15]:
print("Predicted K1: ", " ".join(map(str, predictedK1)))
print("K1 original: ", " ".join(map(str, K1)))
print("\nPredicted K2: ", " ".join(map(str, predictedK2)))
print("K2 original: ", " ".join(map(str, K2)))
print("\nPredicted ID: ", " ".join(map(str, predictedID)))
print("ID original: ", " ".join(map(str, ID)))

Predicted K1:  1 1 1 0 1 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 0 1
K1 original:  1 1 1 0 1 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 0 1

Predicted K2:  1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0
K2 original:  1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0

Predicted ID:  0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1

### Finally, check again our predictions!

In [16]:
print(hamming2(K1, predictedK1))
print(hamming2(K2, predictedK2))
print(hamming2(ID, predictedID))

0
0
0
